In [ ]:
# merge health data and annual measures data for now (pca data later) 

In [1]:
import os
import pandas as pd

In [2]:
# load annual measures
annual_measures = pd.read_csv(os.path.join("..", "processed", "collapsed_measures", "LA_zipcode_measures_all_years.csv"))
annual_measures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4626 entries, 0 to 4625
Columns: 171 entries, YEAR to HEATWAVE_CNT
dtypes: float64(144), int64(27)
memory usage: 6.0 MB


In [5]:
# load health data (just mortality for now)
mortality = pd.read_csv(os.path.join("..", "processed", "health_data", "mortality_rates_catchment_correct_final.csv"))
mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3936 entries, 0 to 3935
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Year                3936 non-null   int64  
 1   Zipcode             3936 non-null   int64  
 2   Age_0_4             3936 non-null   float64
 3   Age_5_14            3936 non-null   int64  
 4   Age_15_64           3936 non-null   float64
 5   Age_65_84           3936 non-null   float64
 6   Age_85_and_Over     3936 non-null   float64
 7   Accidents           3936 non-null   float64
 8   Total               3936 non-null   float64
 9   Alzheimers          3936 non-null   float64
 10  Homicide            3936 non-null   float64
 11  Cerebrovascular     3936 non-null   float64
 12  Cirrhosis           3936 non-null   float64
 13  Respiratory         3936 non-null   float64
 14  Diabetes            3936 non-null   float64
 15  Heart               3936 non-null   float64
 16  Hypert

In [20]:
# compare zipcodes and years used for each
measures_zips = annual_measures.loc[annual_measures["YEAR"] == 2003, "ZIPCODE"]
mortality_zips = mortality.loc[mortality["Year"] == 2005, "Zipcode"]

measures_years = annual_measures["YEAR"].unique()
mortality_years = mortality["Year"].unique()


In [21]:
print(measures_zips)
print(measures_years)

0      90001
1      90002
2      90003
3      90004
4      90005
       ...  
252    91790
253    91791
254    91792
255    91801
256    91803
Name: ZIPCODE, Length: 257, dtype: int64
[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020]


In [22]:
print(mortality_zips)
print(mortality_years)

0      90001
1      90002
2      90003
3      90004
4      90005
       ...  
241    91790
242    91791
243    91792
244    91801
245    91803
Name: Zipcode, Length: 246, dtype: int64
[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2020]


In [23]:
# filter annual measures to only contain moratality zips and mortality years
annual_measures_filtered = annual_measures[annual_measures.loc[:, "ZIPCODE"].isin(mortality_zips) & annual_measures.loc[:, "YEAR"].isin(mortality_years)]
annual_measures_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3936 entries, 514 to 4625
Columns: 171 entries, YEAR to HEATWAVE_CNT
dtypes: float64(144), int64(27)
memory usage: 5.2 MB


In [25]:
annual_measures_filtered.reset_index(drop=True, inplace=True)

In [27]:
annual_measures_filtered.tail()

,YEAR,ZIPCODE,DAILY_AVG_TEMP_Inf_0,DAILY_AVG_TEMP_0_1,DAILY_AVG_TEMP_1_2,DAILY_AVG_TEMP_2_3,DAILY_AVG_TEMP_3_4,DAILY_AVG_TEMP_4_5,DAILY_AVG_TEMP_5_6,DAILY_AVG_TEMP_6_7,...,DIURNAL_TEMP_RNG_32_33,DIURNAL_TEMP_RNG_33_34,DIURNAL_TEMP_RNG_34_35,DIURNAL_TEMP_RNG_35_36,DIURNAL_TEMP_RNG_36_37,DIURNAL_TEMP_RNG_37_38,DIURNAL_TEMP_RNG_38_39,DIURNAL_TEMP_RNG_39_40,DIURNAL_TEMP_RNG_40_Inf,HEATWAVE_CNT
3931,2020,91790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,6.521943
3932,2020,91791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5.923969
3933,2020,91792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5.058040
3934,2020,91801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5.578315
3935,2020,91803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5.483635


In [33]:
mortality.rename(columns={"Year":"YEAR", "Zipcode":"ZIPCODE"}, inplace=True)

In [38]:
# merge the two dfs!
measures_mortality = annual_measures_filtered.merge(mortality, on=["YEAR", "ZIPCODE"])

In [39]:
measures_mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3936 entries, 0 to 3935
Columns: 191 entries, YEAR to Parkinsons
dtypes: float64(163), int64(28)
memory usage: 5.7 MB


In [40]:
# save 
out_path = os.path.join("..", "processed", "merged", "measures_mortality.csv")
measures_mortality.to_csv(out_path, index=False)